In [47]:
"""01_generate_additional_zoos.ipynb

Goal
- Inspect `./data/Merged zoo.csv` (columns + a few rows).
- Using the existing CNN zoo checkpoints in `./checkpoints`, build additional zoo CSVs for other initializations so they share the same schema as `Merged zoo.csv`.
- We DO NOT modify existing data/code; this notebook only reads and writes new CSVs.

Notes
- The current CNN zoo generator is in `useful/Silu.py`.
- `Double_input_transformer.CustomDataset` uses:
    - `df = pd.read_csv('./data/Merged zoo.csv')`
    - `params_cols = df.columns[17:-2]` as weight columns.
- We will reproduce the same layout for new CSVs to keep `CustomDataset` and the rest of the pipeline usable.
"""

import os
from pathlib import Path
import sys

# Add parent directory to path to import project modules
sys.path.append(str(Path("..").resolve()))

import numpy as np
import pandas as pd
import torch
import ast
from tqdm import tqdm

# Project root (assumes notebook is run from repo root or this path exists)
ROOT = Path("..").resolve()
DATA_DIR = ROOT / "data"
CHECKPOINTS_DIR = ROOT / "checkpoints"
OUTPUT_DIR = DATA_DIR / "generated_zoos"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

MERGED_ZOO_PATH = DATA_DIR / "Merged zoo.csv"

print(f"Project root: {ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Checkpoints directory: {CHECKPOINTS_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Merged zoo path: {MERGED_ZOO_PATH}")

Project root: /home/aymen/Documents/GitHub/Federated-Continual-learning-/New
Data directory: /home/aymen/Documents/GitHub/Federated-Continual-learning-/New/data
Checkpoints directory: /home/aymen/Documents/GitHub/Federated-Continual-learning-/New/checkpoints
Output directory: /home/aymen/Documents/GitHub/Federated-Continual-learning-/New/data/generated_zoos
Merged zoo path: /home/aymen/Documents/GitHub/Federated-Continual-learning-/New/data/Merged zoo.csv


In [48]:
# Cell 2: Inspect Merged zoo.csv structure
print("=== Inspecting Merged zoo.csv ===")

# Read first few rows to understand structure
df_sample = pd.read_csv(MERGED_ZOO_PATH, nrows=5)
print("Shape:", df_sample.shape)
print("\nColumns:")
#for i, col in enumerate(df_sample.columns):
#    print(f"{i:2d}: {col}")

print("\nFirst few rows:")
print(df_sample.head())

# Identify key column groups
meta_cols = df_sample.columns[:17]  # First 17 columns are metadata
weight_cols = df_sample.columns[17:-2]  # Weight parameters
final_cols = df_sample.columns[-2:]  # Last 2 columns

#print(f"\nMeta columns: {len(meta_cols)}")
#print(f"Weight columns: {len(weight_cols)}")  
#print(f"Final columns: {len(final_cols)}")

# Check activation columns (should be binary indicators)
activation_cols = [col for col in meta_cols if col in ["silu", "gelu", "relu", "leakyrelu", "sigmoid", "tanh"]]
print(f"\nActivation columns found: {activation_cols}")

# Check unique labels and epochs
print(f"\nUnique labels: {sorted(df_sample['label'].unique())}")
print(f"Unique epochs: {sorted(df_sample['epoch'].unique())}")

=== Inspecting Merged zoo.csv ===
Shape: (5, 2483)

Columns:

First few rows:
    label  0  1  2  3  4  5  6  7  8  ...  bias 2456  bias 2457  bias 2458  \
0  [0, 1]  1  1  0  0  0  0  0  0  0  ...  -0.126967  -0.043210  -0.231192   
1  [0, 2]  1  0  1  0  0  0  0  0  0  ...  -0.173608  -0.093011   0.303516   
2  [0, 3]  1  0  0  1  0  0  0  0  0  ...  -0.215478  -0.561090  -0.173014   
3  [0, 4]  1  0  0  0  1  0  0  0  0  ...   0.152755   0.408451  -0.191886   
4  [0, 5]  1  0  0  0  0  1  0  0  0  ...  -0.212532  -0.178356   0.269947   

   bias 2459  bias 2460  bias 2461  bias 2462  bias 2463   Accuracy  epoch  
0   0.342562  -0.112175   0.279830  -0.136550  -0.327724  98.643333     36  
1   0.819489  -0.519285  -0.212039   0.197068  -0.362670  98.906667     36  
2  -0.151766  -0.098772   0.597230   0.088529  -0.206130  98.376667     36  
3   0.263275  -0.203316   0.037706  -0.203713  -0.283887  98.911667     36  
4  -0.030196  -0.511167  -0.002527   0.088270  -0.004224  98.868333 

In [49]:
# Cell 3: Define CNN architecture directly (avoiding avalanche dependency)
print("=== Defining CNN Architecture ===")

# Define CNN class directly to avoid external dependencies
import torch.nn as nn

class CNN(nn.Module):
    def __init__(
        self,
        channels_in,
        nlin="leakyrelu",
        dropout=0.0,
        init_type="uniform",
    ):
        super().__init__()
        # init module list
        self.module_list = nn.ModuleList()
        ### ASSUMES 28x28 image size
        ## compose layer 1
        self.module_list.append(nn.Conv2d(channels_in, 8, 5))
        self.module_list.append(nn.MaxPool2d(2, 2))
        self.module_list.append(self.get_nonlin(nlin))
        # apply dropout
        if dropout > 0:
            self.module_list.append(nn.Dropout(dropout))
        ## compose layer 2
        self.module_list.append(nn.Conv2d(8, 6, 5))
        self.module_list.append(nn.MaxPool2d(2, 2))
        self.module_list.append(self.get_nonlin(nlin))
        ## add dropout
        if dropout > 0:
            self.module_list.append(nn.Dropout(dropout))
        ## compose layer 3
        self.module_list.append(nn.Conv2d(6, 4, 2))
        self.module_list.append(self.get_nonlin(nlin))
        ## add flatten layer
        self.module_list.append(nn.Flatten())
        ## add linear layer 1
        self.module_list.append(nn.Linear(3 * 3 * 4, 20))
        self.module_list.append(self.get_nonlin(nlin))
        ## add dropout
        if dropout > 0:
            self.module_list.append(nn.Dropout(dropout))
        ## add linear layer 1
        self.module_list.append(nn.Linear(20, 10))

        ### initialize weights with se methods
        self.initialize_weights(init_type)

    def initialize_weights(self, init_type):
        # print("initialze model")
        for m in self.module_list:
            if type(m) == nn.Linear or type(m) == nn.Conv2d:
                if init_type == "xavier_uniform":
                    torch.nn.init.xavier_uniform_(m.weight)
                if init_type == "xavier_normal":
                    torch.nn.init.xavier_normal_(m.weight)
                if init_type == "uniform":
                    torch.nn.init.uniform_(m.weight)
                if init_type == "normal":
                    torch.nn.init.normal_(m.weight)
                if init_type == "kaiming_normal":
                    torch.nn.init.kaiming_normal_(m.weight)
                if init_type == "kaiming_uniform":
                    torch.nn.init.kaiming_uniform_(m.weight)
                # set bias to some small non-zero value
                m.bias.data.fill_(0.01)

    def get_nonlin(self, nlin):
        # apply nonlinearity
        if nlin == "leakyrelu":
            return nn.LeakyReLU()
        if nlin == "relu":
            return nn.ReLU()
        if nlin == "tanh":
            return nn.Tanh()
        if nlin == "sigmoid":
            return nn.Sigmoid()
        if nlin == "silu":
            return nn.SiLU()
        if nlin == "gelu":
            return nn.GELU()

    def forward(self, x):
        # forward prop through module_list
        for layer in self.module_list:
            x = layer(x)
        return x

    def forward_activations(self, x):
        # forward prop through module_list
        activations = []
        for layer in self.module_list:
            x = layer(x)
            if (
                isinstance(layer, nn.Tanh)
                or isinstance(layer, nn.Sigmoid)
                or isinstance(layer, nn.ReLU)
                or isinstance(layer, nn.LeakyReLU)
                or isinstance(layer, nn.SiLU)
                or isinstance(layer, nn.GELU)
            ):
                activations.append(x)
        return x, activations

print("CNN class defined successfully")
print("Available initializations:")
print("- xavier_uniform")
print("- xavier_normal") 
print("- uniform")
print("- normal")
print("- kaiming_normal")
print("- kaiming_uniform")
print("\nAvailable activations:")
print("- leakyrelu")
print("- relu")
print("- tanh")
print("- sigmoid")
print("- silu")
print("- gelu")

# The current Merged zoo.csv uses "kaiming_uniform" based on the document
# We need to generate zoos for the other 5 initializations

=== Defining CNN Architecture ===
CNN class defined successfully
Available initializations:
- xavier_uniform
- xavier_normal
- uniform
- normal
- kaiming_normal
- kaiming_uniform

Available activations:
- leakyrelu
- relu
- tanh
- sigmoid
- silu
- gelu


In [50]:
# Cell 4: Check directory access and browse checkpoints
print("=== Checking Directory Access and Browsing Checkpoints ===")

import os
import stat

def check_directory_access(path, name):
    """Check if we have read access to a directory"""
    try:
        if not path.exists():
            print(f"❌ {name} does not exist: {path}")
            return False
        
        if not path.is_dir():
            print(f"❌ {name} is not a directory: {path}")
            return False
        
        # Check read permissions
        if os.access(path, os.R_OK):
            print(f"✅ {name} is readable: {path}")
        else:
            print(f"❌ {name} is not readable: {path}")
            return False
        
        # Try to list contents
        try:
            contents = list(path.iterdir())
            print(f"✅ {name} contains {len(contents)} items")
            return True
        except PermissionError as e:
            print(f"❌ Permission error accessing {name}: {e}")
            return False
        except Exception as e:
            print(f"❌ Error accessing {name}: {e}")
            return False
            
    except Exception as e:
        print(f"❌ Unexpected error checking {name}: {e}")
        return False

# Check all important directories
print("Checking directory access...")
data_ok = check_directory_access(DATA_DIR, "Data directory")
checkpoints_ok = check_directory_access(CHECKPOINTS_DIR, "Checkpoints directory")
output_ok = check_directory_access(OUTPUT_DIR, "Output directory")

if not (data_ok and checkpoints_ok):
    print("\n❌ Some directories are not accessible. Trying alternative approaches...")
    
    # Try alternative paths
    alt_data_dir = Path("./data").resolve()
    alt_checkpoints_dir = Path("./checkpoints").resolve()
    
    print(f"\nTrying alternative paths:")
    print(f"Alternative data: {alt_data_dir}")
    print(f"Alternative checkpoints: {alt_checkpoints_dir}")
    
    if alt_data_dir.exists():
        DATA_DIR = alt_data_dir
        print(f"✅ Using alternative data directory: {DATA_DIR}")
        data_ok = True
    
    if alt_checkpoints_dir.exists():
        CHECKPOINTS_DIR = alt_checkpoints_dir
        print(f"✅ Using alternative checkpoints directory: {CHECKPOINTS_DIR}")
        checkpoints_ok = True

# Browse checkpoints directory structure if accessible
if checkpoints_ok:
    print(f"\n=== Browsing Checkpoints Directory Structure ===")
    
    def browse_directory(path, indent=0, max_depth=3, current_depth=0):
        """Recursively browse directory structure"""
        if current_depth >= max_depth:
            return
        
        try:
            items = sorted(path.iterdir())
            for item in items:
                prefix = "  " * indent
                if item.is_dir():
                    print(f"{prefix}📁 {item.name}/")
                    # Show some stats if this looks like a checkpoint directory
                    if any(x in item.name for x in ['[', ']', 'gelu', 'relu', 'silu', 'tanh', 'sigmoid', 'leakyrelu']):
                        try:
                            subitems = list(item.iterdir())
                            print(f"{prefix}   ({len(subitems)} items)")
                            
                            # Look for .pth files
                            pth_files = list(item.glob("*.pth"))
                            if pth_files:
                                print(f"{prefix}   📄 {len(pth_files)} .pth files")
                                
                        except:
                            print(f"{prefix}   (access denied)")
                    
                    # Recurse into subdirectories
                    browse_directory(item, indent + 1, max_depth, current_depth + 1)
                else:
                    if item.suffix in ['.pth', '.csv', '.txt']:
                        print(f"{prefix}📄 {item.name}")
                    
        except PermissionError:
            print(f"  " * indent + "❌ Permission denied")
        except Exception as e:
            print(f"  " * indent + f"❌ Error: {e}")
    
    browse_directory(CHECKPOINTS_DIR)
    
    # Look for specific checkpoint patterns
    print(f"\n=== Looking for Checkpoint Patterns ===")
    
    # Find all .pth files
    pth_files = list(CHECKPOINTS_DIR.rglob("*.pth"))
    print(f"Found {len(pth_files)} .pth files total")
    
    if pth_files:
        print(f"Sample checkpoint files:")
        for pth in pth_files[:10]:  # Show first 10
            rel_path = pth.relative_to(CHECKPOINTS_DIR)
            print(f"  {rel_path}")
    
    # Look for directory patterns that match our expected structure
    print(f"\n=== Expected Directory Structure Analysis ===")
    
    class_dirs = [d for d in CHECKPOINTS_DIR.iterdir() 
                  if d.is_dir() and d.name.startswith('[') and d.name.endswith(']')]
    
    print(f"Found {len(class_dirs)} class configuration directories")
    
    if class_dirs:
        print(f"Class configurations:")
        for class_dir in class_dirs[:5]:  # Show first 5
            print(f"  {class_dir.name}")
            
            # Look for activation subdirectories
            try:
                activ_dirs = [d for d in class_dir.iterdir() if d.is_dir()]
                activations = [d.name for d in activ_dirs 
                              if d.name in ['gelu', 'relu', 'silu', 'tanh', 'sigmoid', 'leakyrelu']]
                if activations:
                    print(f"    Activations: {', '.join(activations)}")
            except:
                print(f"    (cannot access subdirectories)")

else:
    print("❌ Cannot access checkpoints directory. Please check permissions.")
    print("Suggestions:")
    print("1. Run the notebook from the project root directory")
    print("2. Check that the checkpoints directory exists and is readable")
    print("3. Try running: chmod -R +r /path/to/checkpoints (if you have permissions)")

print(f"\nFinal directory paths:")
print(f"Data: {DATA_DIR}")
print(f"Checkpoints: {CHECKPOINTS_DIR}")
print(f"Output: {OUTPUT_DIR}")

=== Checking Directory Access and Browsing Checkpoints ===
Checking directory access...
✅ Data directory is readable: /home/aymen/Documents/GitHub/Federated-Continual-learning-/New/data
✅ Data directory contains 6 items
✅ Checkpoints directory is readable: /home/aymen/Documents/GitHub/Federated-Continual-learning-/New/checkpoints
✅ Checkpoints directory contains 1013 items
✅ Output directory is readable: /home/aymen/Documents/GitHub/Federated-Continual-learning-/New/data/generated_zoos
✅ Output directory contains 0 items

=== Browsing Checkpoints Directory Structure ===
📁 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]/
   (6 items)
  📁 gelu/
     (8 items)
     📄 7 .pth files
    📄 checkpoint epoch 11.pth
    📄 checkpoint epoch 16.pth
    📄 checkpoint epoch 21.pth
    📄 checkpoint epoch 26.pth
    📄 checkpoint epoch 31.pth
    📄 checkpoint epoch 36.pth
    📄 checkpoint epoch 40.pth
    📁 metrics/
  📁 leakyrelu/
     (8 items)
     📄 7 .pth files
    📄 checkpoint epoch 11.pth
    📄 checkpoint epoch 16.

In [51]:
# Cell 5: Discover checkpoint configurations with robust error handling
print("=== Discovering Checkpoint Configurations ===")

def discover_checkpoint_configurations(checkpoints_path):
    """
    Discover all checkpoint configurations with comprehensive error handling
    """
    
    configurations = []
    
    if not checkpoints_path.exists():
        print(f"❌ Checkpoints directory does not exist: {checkpoints_path}")
        return configurations
    
    print(f"Scanning: {checkpoints_path}")
    
    # First level: Look for class configuration directories [0,1,2,3,4], etc.
    try:
        class_dirs = []
        for item in checkpoints_path.iterdir():
            if not item.is_dir():
                continue
                
            # Check if it matches the pattern [0, 1, 2, 3, 4] or similar
            if item.name.startswith('[') and item.name.endswith(']'):
                try:
                    # Try to parse as a list
                    classes_subset = ast.literal_eval(item.name)
                    if isinstance(classes_subset, list) and len(classes_subset) == 5:
                        class_dirs.append(item)
                        print(f"  ✅ Found class config: {item.name}")
                    else:
                        print(f"  ⚠️  Skipping invalid class config: {item.name}")
                except:
                    print(f"  ⚠️  Cannot parse class config: {item.name}")
            else:
                print(f"  ⚠️  Skipping non-class directory: {item.name}")
        
        print(f"Found {len(class_dirs)} valid class configuration directories")
        
    except PermissionError as e:
        print(f"❌ Permission error scanning checkpoints: {e}")
        return configurations
    except Exception as e:
        print(f"❌ Error scanning checkpoints: {e}")
        return configurations
    
    # Second level: For each class config, look for activation directories
    for class_dir in class_dirs:
        try:
            print(f"\n  Scanning class config: {class_dir.name}")
            
            activation_dirs = []
            for item in class_dir.iterdir():
                if not item.is_dir():
                    continue
                
                # Check if it's a valid activation
                valid_activations = ["silu", "gelu", "relu", "leakyrelu", "sigmoid", "tanh"]
                if item.name in valid_activations:
                    activation_dirs.append(item)
                    print(f"    ✅ Found activation: {item.name}")
                else:
                    print(f"    ⚠️  Skipping non-activation directory: {item.name}")
            
            print(f"    Found {len(activation_dirs)} activation directories")
            
        except PermissionError as e:
            print(f"    ❌ Permission error for {class_dir.name}: {e}")
            continue
        except Exception as e:
            print(f"    ❌ Error scanning {class_dir.name}: {e}")
            continue
        
        # Third level: For each activation, look for initialization directories
        for activation_dir in activation_dirs:
            try:
                print(f"\n    Scanning activation: {activation_dir.name}")
                
                init_dirs = []
                for item in activation_dir.iterdir():
                    if not item.is_dir():
                        continue
                    
                    # Check if it's a valid initialization
                    valid_inits = ["xavier_uniform", "uniform", "kaiming_normal", 
                                  "xavier_normal", "normal", "kaiming_uniform"]
                    if item.name in valid_inits:
                        init_dirs.append(item)
                        print(f"      ✅ Found initialization: {item.name}")
                    else:
                        print(f"      ⚠️  Skipping non-init directory: {item.name}")
                
                print(f"      Found {len(init_dirs)} initialization directories")
                
            except PermissionError as e:
                print(f"      ❌ Permission error for {activation_dir.name}: {e}")
                continue
            except Exception as e:
                print(f"      ❌ Error scanning {activation_dir.name}: {e}")
                continue
            
            # Fourth level: For each initialization, look for checkpoint files
            for init_dir in init_dirs:
                try:
                    print(f"\n      Scanning initialization: {init_dir.name}")
                    
                    # Look for .pth files
                    pth_files = list(init_dir.glob("*.pth"))
                    print(f"        Found {len(pth_files)} .pth files")
                    
                    # Look for tracking directories (experiment results)
                    tracking_dirs = []
                    for item in init_dir.iterdir():
                        if item.is_dir() and ('Tracking' in item.name or 'tracking' in item.name):
                            tracking_dirs.append(item)
                    
                    if tracking_dirs:
                        print(f"        Found {len(tracking_dirs)} tracking directories")
                    
                    # Only add configuration if we have files
                    if pth_files or tracking_dirs:
                        try:
                            classes_subset = ast.literal_eval(class_dir.name)
                            config = {
                                'classes': classes_subset,
                                'activation': activation_dir.name,
                                'init': init_dir.name,
                                'path': init_dir,
                                'checkpoint_files': pth_files,
                                'tracking_dirs': tracking_dirs,
                                'config_id': f"{classes_subset}_{activation_dir.name}_{init_dir.name}"
                            }
                            configurations.append(config)
                            print(f"        ✅ Added configuration: {config['config_id']}")
                        except Exception as e:
                            print(f"        ❌ Error creating config: {e}")
                    else:
                        print(f"        ⚠️  No checkpoint files found, skipping")
                        
                except PermissionError as e:
                    print(f"        ❌ Permission error for {init_dir.name}: {e}")
                    continue
                except Exception as e:
                    print(f"        ❌ Error scanning {init_dir.name}: {e}")
                    continue
    
    print(f"\n=== Discovery Complete ===")
    print(f"Total configurations found: {len(configurations)}")
    
    if configurations:
        print(f"\nConfiguration summary:")
        init_counts = {}
        activ_counts = {}
        
        for config in configurations:
            init = config['init']
            activ = config['activation']
            
            init_counts[init] = init_counts.get(init, 0) + 1
            activ_counts[activ] = activ_counts.get(activ, 0) + 1
        
        print(f"By initialization:")
        for init, count in sorted(init_counts.items()):
            print(f"  {init}: {count} configs")
        
        print(f"By activation:")
        for activ, count in sorted(activ_counts.items()):
            print(f"  {activ}: {count} configs")
    
    return configurations

# Discover configurations
checkpoints = discover_checkpoint_configurations(CHECKPOINTS_DIR)

if not checkpoints:
    print(f"\n❌ No checkpoint configurations found!")
    print(f"Troubleshooting suggestions:")
    print(f"1. Check that you're running from the correct directory")
    print(f"2. Verify the checkpoints directory structure matches the expected pattern")
    print(f"3. Check file permissions: ls -la {CHECKPOINTS_DIR}")
    print(f"4. Try running: find {CHECKPOINTS_DIR} -name '*.pth' | head -10")
else:
    print(f"\n✅ Successfully discovered {len(checkpoints)} configurations!")
    
    # Show a few sample configurations
    print(f"\nSample configurations:")
    for i, config in enumerate(checkpoints[:3]):
        print(f"  {i+1}. {config['config_id']}")
        print(f"     Classes: {config['classes']}")
        print(f"     Checkpoints: {len(config['checkpoint_files'])}")
        print(f"     Tracking dirs: {len(config['tracking_dirs'])}")

print(f"\nReady to proceed with zoo generation!")

=== Discovering Checkpoint Configurations ===
Scanning: /home/aymen/Documents/GitHub/Federated-Continual-learning-/New/checkpoints
  ⚠️  Skipping invalid class config: [1, 7, 9]
  ✅ Found class config: [2, 3, 4, 5, 6]
  ✅ Found class config: [0, 2, 4, 5, 8]
  ✅ Found class config: [1, 2, 4, 5, 8]
  ✅ Found class config: [0, 1, 2, 8, 9]
  ⚠️  Skipping invalid class config: [0, 1, 2, 3, 4, 5, 6, 8, 9]
  ⚠️  Skipping invalid class config: [0, 2, 3, 5, 6, 7, 9]
  ✅ Found class config: [1, 2, 5, 7, 8]
  ⚠️  Skipping invalid class config: [0, 2, 4, 5, 6, 9]
  ⚠️  Skipping invalid class config: [0, 1, 4, 5, 6, 7, 8]
  ⚠️  Skipping invalid class config: [4, 5, 7, 8]
  ⚠️  Skipping invalid class config: [1, 3, 5, 7]
  ⚠️  Skipping invalid class config: [0, 3, 4, 5, 6, 7, 9]
  ⚠️  Skipping invalid class config: [1, 2, 4, 6, 7, 9]
  ⚠️  Skipping invalid class config: [0, 2, 4, 5, 7, 8, 9]
  ⚠️  Skipping invalid class config: [1, 2, 7, 9]
  ⚠️  Skipping invalid class config: [4, 7, 8, 9]
  ⚠️  Ski

In [52]:
# Cell 5: Load reference CSV to understand exact format
print("=== Loading Reference CSV Format ===")

# Load the full CSV to understand the exact column structure
ref_df = pd.read_csv(MERGED_ZOO_PATH, nrows=1000)  # Load first 1000 rows for reference

print(f"Reference CSV shape: {ref_df.shape}")
print(f"Column count: {len(ref_df.columns)}")

# Verify the column structure matches our understanding
meta_cols = list(ref_df.columns[:17])
weight_cols = list(ref_df.columns[17:-2])
final_cols = list(ref_df.columns[-2:])

print(f"\nMeta columns ({len(meta_cols)}): {meta_cols}")
print(f"Weight columns ({len(weight_cols)}): {weight_cols[:5]}...{weight_cols[-5:]}")
print(f"Final columns ({len(final_cols)}): {final_cols}")

# Check which activation is used in the reference CSV
activation_cols = ["silu", "gelu", "relu", "leakyrelu", "sigmoid", "tanh"]
ref_activations = [col for col in activation_cols if col in ref_df.columns]
print(f"\nActivation columns in reference: {ref_activations}")

# Check data types
print(f"\nData types:")
print(ref_df.dtypes.value_counts())

# Sample a row to see the format
print(f"\nSample row:")
sample_row = ref_df.iloc[0]
for key in ['label', 'epoch', 'Accuracy task1', 'Accuracy task2']:
    if key in sample_row:
        print(f"  {key}: {sample_row[key]}")
        
# Check activation pattern in first few rows
print(f"\nActivation pattern in first 5 rows:")
for i in range(min(5, len(ref_df))):
    row = ref_df.iloc[i]
    active_activations = [act for act in activation_cols if act in row and row[act] == 1.0]
    print(f"  Row {i}: {active_activations}")

=== Loading Reference CSV Format ===
Reference CSV shape: (1000, 2483)
Column count: 2483

Meta columns (17): ['label', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'gelu', 'relu', 'silu', 'tanh', 'sigmoid', 'leakyrelu']
Weight columns (2464): ['weight 0', 'weight 1', 'weight 2', 'weight 3', 'weight 4']...['bias 2459', 'bias 2460', 'bias 2461', 'bias 2462', 'bias 2463']
Final columns (2): ['Accuracy', 'epoch']

Activation columns in reference: ['silu', 'gelu', 'relu', 'leakyrelu', 'sigmoid', 'tanh']

Data types:
float64    2471
int64        11
object        1
Name: count, dtype: int64

Sample row:
  label: [0, 1]
  epoch: 36

Activation pattern in first 5 rows:
  Row 0: ['gelu']
  Row 1: ['gelu']
  Row 2: ['gelu']
  Row 3: ['gelu']
  Row 4: ['gelu']


In [53]:
# Cell 6: Function to flatten model weights
print("=== Weight Flattening Function ===")

def flatten_model_weights(model):
    """Flatten model weights into a 1D array matching the CSV format"""
    weights = []
    
    # Iterate through modules in the same order as Silu.py
    for i, module in enumerate(model.module_list):
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            # Flatten weights and biases separately
            w = module.weight.data.cpu().numpy().flatten()
            b = module.bias.data.cpu().numpy().flatten()
            weights.extend(w)
            weights.extend(b)
    
    return np.array(weights)

def get_weight_column_names():
    """Generate the weight column names based on the CNN architecture"""
    weight_cols = []
    
    # Based on the CNN architecture in Silu.py:
    # Conv2d(1, 8, 5) -> 8*1*5*5 = 200 weights + 8 biases
    for i in range(200):
        weight_cols.append(f"weight {i}")
    for i in range(200, 208):
        weight_cols.append(f"bias {i}")
    
    # Conv2d(8, 6, 5) -> 6*8*5*5 = 1200 weights + 6 biases  
    for i in range(208, 1408):
        weight_cols.append(f"weight {i}")
    for i in range(1408, 1414):
        weight_cols.append(f"bias {i}")
    
    # Conv2d(6, 4, 2) -> 4*6*2*2 = 96 weights + 4 biases
    for i in range(1414, 1510):
        weight_cols.append(f"weight {i}")
    for i in range(1510, 1514):
        weight_cols.append(f"bias {i}")
    
    # Linear(3*3*4, 20) -> 20*36 = 720 weights + 20 biases
    for i in range(1514, 2234):
        weight_cols.append(f"weight {i}")
    for i in range(2234, 2254):
        weight_cols.append(f"bias {i}")
    
    # Linear(20, 10) -> 10*20 = 200 weights + 10 biases
    for i in range(2254, 2454):
        weight_cols.append(f"weight {i}")
    for i in range(2454, 2464):
        weight_cols.append(f"bias {i}")
    
    return weight_cols

# Test the weight column generation
test_weight_cols = get_weight_column_names()
print(f"Generated {len(test_weight_cols)} weight column names")
print(f"First few: {test_weight_cols[:5]}")
print(f"Last few: {test_weight_cols[-5:]}")

# Verify it matches the reference
if len(weight_cols) == len(test_weight_cols):
    print("✓ Weight column count matches reference")
else:
    print(f"✗ Weight column count mismatch: {len(test_weight_cols)} vs {len(weight_cols)}")

=== Weight Flattening Function ===
Generated 2464 weight column names
First few: ['weight 0', 'weight 1', 'weight 2', 'weight 3', 'weight 4']
Last few: ['bias 2459', 'bias 2460', 'bias 2461', 'bias 2462', 'bias 2463']
✓ Weight column count matches reference


In [54]:
# Cell 7: Function to load checkpoint and extract metadata
print("=== Checkpoint Loading Function ===")

def load_checkpoint_data(checkpoint_info, target_epoch=None):
    """Load checkpoint and extract weights with metadata"""
    
    # Create model with same architecture
    model = CNN(1, checkpoint_info['activation'], 0, checkpoint_info['init'])
    
    # Find the appropriate checkpoint file
    checkpoint_files = checkpoint_info['checkpoint_files']
    
    if target_epoch is not None:
        # Look for specific epoch
        target_file = None
        for cf in checkpoint_files:
            if f"epoch {target_epoch}" in cf.name:
                target_file = cf
                break
        if target_file is None:
            # Fallback to main checkpoint
            target_file = checkpoint_info['path'] / "checkpoint.pth"
    else:
        # Use the main checkpoint file
        target_file = checkpoint_info['path'] / "checkpoint.pth"
    
    if not target_file.exists():
        return None
    
    # Load checkpoint
    try:
        checkpoint = torch.load(target_file, map_location='cpu')
        model.load_state_dict(checkpoint['model_state_dict'])
        
        # Load training metrics if available
        metrics = {}
        metrics_file = checkpoint_info['path'] / "Test Accuracy IID.npy"
        if metrics_file.exists():
            test_acc = np.load(metrics_file)
            if len(test_acc) > 0:
                metrics['Accuracy task1'] = test_acc[-1]  # Final accuracy
        
        # Extract weights
        weights = flatten_model_weights(model)
        
        return {
            'weights': weights,
            'metrics': metrics,
            'epoch': checkpoint.get('epoch', 0),
            'checkpoint_file': str(target_file)
        }
        
    except Exception as e:
        print(f"Error loading {target_file}: {e}")
        return None

# Test loading one checkpoint
if checkpoints:
    test_cp = checkpoints[0]
    print(f"Testing with: {test_cp['classes']}, {test_cp['activation']}, {test_cp['init']}")
    
    result = load_checkpoint_data(test_cp)
    if result:
        print(f"✓ Loaded checkpoint with epoch {result['epoch']}")
        print(f"  Weights shape: {result['weights'].shape}")
        print(f"  Metrics: {result['metrics']}")
    else:
        print("✗ Failed to load checkpoint")

=== Checkpoint Loading Function ===


In [55]:
# Cell 8: Generate zoo CSV for each initialization
print("=== Generating Additional Zoo CSVs ===")

def generate_zoo_csv(initialization, checkpoints_list):
    """Generate a zoo CSV for a specific initialization"""
    
    print(f"\nGenerating zoo for initialization: {initialization}")
    
    # Filter checkpoints for this initialization
    init_checkpoints = [cp for cp in checkpoints_list if cp['init'] == initialization]
    print(f"Found {len(init_checkpoints)} configurations")
    
    # Prepare column structure
    all_cols = (list(meta_cols) + list(weight_cols) + list(final_cols))
    
    rows = []
    
    for cp in tqdm(init_checkpoints, desc=f"Processing {initialization}"):
        classes_subset = cp['classes']
        activation = cp['activation']
        
        # Create label string matching reference format
        label_str = str(classes_subset)
        
        # Try to load data for different epochs (11, 16, 21, 26, 31, 36)
        target_epochs = [11, 16, 21, 26, 31, 36]
        
        for target_epoch in target_epochs:
            # Load checkpoint data
            result = load_checkpoint_data(cp, target_epoch)
            
            if result is None:
                continue
                
            # Create row with metadata
            row = {}
            
            # Meta columns (first 17)
            row['label task 1'] = label_str
            row['index'] = 0  # Will be updated later
            row['Accuracy task1'] = result['metrics'].get('Accuracy task1', np.nan)
            row['label task 2'] = ""  # Empty for now
            row['Accuracy task2'] = np.nan
            
            # Activation indicators (only one should be 1.0)
            for act in activation_cols:
                row[act] = 1.0 if act == activation else 0.0
            
            # Add remaining meta columns to match count (if any)
            for i, col in enumerate(meta_cols):
                if col not in row:
                    row[col] = np.nan
            
            # Weight columns
            weights = result['weights']
            for i, col in enumerate(weight_cols):
                if i < len(weights):
                    row[col] = weights[i]
                else:
                    row[col] = np.nan
            
            # Final columns
            row['Loader Set'] = 'train'
            row['Reconstructed Accuracy ID'] = np.nan
            row['Actual Accuracy'] = result['metrics'].get('Accuracy task1', np.nan)
            row['Reconstructed Accuracy OOD'] = np.nan
            row['Transformer Loss'] = np.nan
            row['lr'] = 0.05  # Default learning rate from Silu.py
            row['epochCNN'] = result['epoch']
            row['ActivationCNN'] = activation
            
            rows.append(row)
    
    if not rows:
        print(f"No data generated for {initialization}")
        return None
    
    # Create DataFrame
    df = pd.DataFrame(rows)
    
    # Update index column
    df['index'] = range(len(df))
    
    # Ensure column order matches reference
    df = df.reindex(columns=all_cols, fill_value=np.nan)
    
    print(f"Generated {len(df)} rows for {initialization}")
    
    return df

# Generate CSVs for initializations other than 'kaiming_uniform' (already in reference)
target_initializations = ['xavier_uniform', 'uniform', 'kaiming_normal', 'xavier_normal', 'normal']

generated_dfs = {}
for init in target_initializations:
    df = generate_zoo_csv(init, checkpoints)
    if df is not None:
        generated_dfs[init] = df
        
        # Save to file
        output_path = OUTPUT_DIR / f"Merged_zoo_{init.replace('_', '_')}.csv"
        df.to_csv(output_path, index=False)
        print(f"Saved to: {output_path}")

print(f"\nGenerated {len(generated_dfs)} additional zoo CSVs")

=== Generating Additional Zoo CSVs ===

Generating zoo for initialization: xavier_uniform
Found 0 configurations


Processing xavier_uniform: 0it [00:00, ?it/s]


No data generated for xavier_uniform

Generating zoo for initialization: uniform
Found 0 configurations


Processing uniform: 0it [00:00, ?it/s]


No data generated for uniform

Generating zoo for initialization: kaiming_normal
Found 0 configurations


...

No data generated for kaiming_normal

Generating zoo for initialization: xavier_normal
Found 0 configurations


Processing xavier_normal: 0it [00:00, ?it/s]


No data generated for xavier_normal

Generating zoo for initialization: normal
Found 0 configurations


Processing normal: 0it [00:00, ?it/s]

No data generated for normal

Generated 0 additional zoo CSVs


In [56]:
# Cell 9: Validation and Summary
print("=== Validation and Summary ===")

# Load and validate one generated CSV
if generated_dfs:
    sample_init = list(generated_dfs.keys())[0]
    sample_df = generated_dfs[sample_init]
    
    print(f"Sample generated CSV ({sample_init}):")
    print(f"Shape: {sample_df.shape}")
    print(f"Columns: {len(sample_df.columns)}")
    
    # Compare with reference
    print(f"\nColumn comparison with reference:")
    print(f"Reference columns: {len(ref_df.columns)}")
    print(f"Generated columns: {len(sample_df.columns)}")
    print(f"Columns match: {list(ref_df.columns) == list(sample_df.columns)}")
    
    # Check data distribution
    print(f"\nData distribution:")
    print(f"Unique labels: {sample_df['label task 1'].nunique()}")
    print(f"Unique epochs: {sorted(sample_df['epochCNN'].unique())}")
    
    # Check activation distribution
    for act in activation_cols:
        if act in sample_df.columns:
            count = (sample_df[act] == 1.0).sum()
            print(f"{act}: {count} rows")
    
    # Sample a few rows
    print(f"\nSample rows:")
    display(sample_df[['label task 1', 'epochCNN', 'ActivationCNN', 'Accuracy task1']].head())

# Summary of all generated files
print(f"\n=== Summary ===")
print(f"Generated zoo CSVs:")
for init, df in generated_dfs.items():
    print(f"  {init}: {len(df)} rows")
    
print(f"\nOutput directory: {OUTPUT_DIR}")
print("Files created:")
for file_path in OUTPUT_DIR.glob("Merged_zoo_*.csv"):
    size_mb = file_path.stat().st_size / (1024 * 1024)
    print(f"  {file_path.name}: {size_mb:.1f} MB")

print(f"\n✓ Notebook 01 complete! Ready to use with CustomDataset by changing df_path.")

=== Validation and Summary ===

=== Summary ===
Generated zoo CSVs:

Output directory: /home/aymen/Documents/GitHub/Federated-Continual-learning-/New/data/generated_zoos
Files created:

✓ Notebook 01 complete! Ready to use with CustomDataset by changing df_path.
